# Автобрея, ДЗ1. Парсер. Катя Шерстнева
Анализ тональности отзывов смотри во второй части

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import regex as re
import pandas as pd

### Парсим отзывы на книги

Сначала пройдемся по главной страничке отзывов с книгами и соберем ссылки на странички с отзывами для каждой книги

In [2]:
def parse(main_link=str, n_pages=int):
  list_of_urls = []

  for page in tqdm(range(1, n_pages)):
    response = requests.get(f"{main_link}?page={page}")
    soup = BeautifulSoup(response.text, 'html.parser')

    post = soup.find_all('div', {'class': 'item_links'})

    for p in post:

      url = re.search('<a href="(.*?)" title', str(p))
      list_of_urls.append('https://vseotzyvy.ru/'+ url.group(1))

  points = [] # оценки
  rew = [] # текст отзыва

  for url in tqdm(list_of_urls):

      r_pos = requests.get(url)

      soup = BeautifulSoup(r_pos.text, 'html.parser')

      stars = soup.find_all('div', {'class': 'rev_rate_box'})
      for el in stars:
        st = re.search('<span class="bold" itemprop="ratingValue">(\d)</span>', str(el)).group(1)
        points.append(st)

      texts = soup.find_all('span', {'itemprop':'reviewBody'})
      for t in texts:
        clean_text = re.search('<span itemprop="reviewBody">(.*?)</span>', str(t))
        if clean_text is None:
          pass
        else:
          clean_text = clean_text.group(1)
        rew.append(clean_text)

  return points, rew

In [ ]:
journals = parse('https://vseotzyvy.ru/category/167/gazetyi-zhurnalyi', 11)

100%|██████████| 150/150 [01:31<00:00,  1.65it/s]


In [ ]:
from collections import Counter
Counter(journals[0])

Counter({'4': 115, '5': 436, '3': 52, '1': 29, '2': 17})

In [ ]:
books = parse('https://vseotzyvy.ru/category/166/knigi', 67)

In [ ]:
def parced_to_df(points=list, rew=list):

  df = pd.DataFrame(columns=['text', 'label'])

  c=0

  for p, r in zip(points, rew):
    if r is not None:
      if int(p)==5:
        df.at [c, 'text'] = r
        df.at [c, 'label'] = 1

      if int(p)<3:
        df.at [c, 'text'] = r
        df.at [c, 'label'] = 0
    c+=1
  return df

In [ ]:
df_b = parced_to_df(books)
df_j = parced_to_df(journals[0], journals[1])

df_0 = df_b[df_b['label'] == 0].sample(n=70, random_state=42)
df_1 = df_b[df_b['label'] == 1].sample(n=70, random_state=42)

df_2 = df_j[df_j['label'] == 0].sample(n=33, random_state=42)
df_3 = df_j[df_j['label'] == 1].sample(n=33, random_state=42)

# Объединяем выборки
df_final = pd.concat([df_0, df_1, df_2, df_3])
df_final = df_final.reset_index(drop=True)

In [ ]:
df_final.to_csv('reviews.csv')